# Part A

In [7]:
import sys
import json
import random
sys.version

'3.7.4 (default, Oct 15 2019, 22:29:14) \n[GCC 7.4.0]'

In [5]:
from py2neo import Graph
from neo4j import GraphDatabase
NEO4J_URI="bolt://localhost:7687"
SC_JSON = "semantic-scholar-raw/s2-corpus-slim.json"
SC_JSON_GITHUB = "https://raw.githubusercontent.com/diegoquintanav/od-lab-1/master/notebooks/semantic-scholar-raw/s2-corpus-slim.json"

graph = Graph(NEO4J_URI)
gdb = GraphDatabase.driver(uri=NEO4J_URI, auth=None)

In [4]:
from faker import Faker
from faker.providers import lorem
fake = Faker()
fake.seed_instance(42)
fake.add_provider(lorem)

http://s2-public-api-prod.us-west-2.elasticbeanstalk.com/corpus/

Download a part of the corpus with the `aws` command line interface, like

```aws s3 cp --no-sign-request s3://ai2-s2-research-public/open-corpus/2018-05-03/s2-corpus-00.gz .```

or

```wget https://s3-us-west-2.amazonaws.com/ai2-s2-research-public/open-corpus/2018-05-03/s2-corpus-00.gz```

And then `gunzip` it.

In [31]:
!ls semantic-scholar-raw/

license.txt   s2-corpus-00	    s2-corpus-clean.tmp
manifest.txt  s2-corpus-clean.json  s2-corpus-slim.json


In [32]:
!wc -l semantic-scholar-raw/s2-corpus-00

1000000 semantic-scholar-raw/s2-corpus-00


Filter out those lines that have empty entities, empty inCitations, and empty outCitations

In [23]:
!grep -v -e '"entities":\[\]' -e '"inCitations":\[\]' -e '"outCitations":\[\]' semantic-scholar-raw/s2-corpus-00 > semantic-scholar-raw/s2-corpus-clean.tmp
!wc -l semantic-scholar-raw/s2-corpus-clean.tmp

212901 semantic-scholar-raw/s2-corpus-clean.tmp


Filter out those rows with empty attributes

In [28]:
!grep -v -e '"journalName":""' -e '"title":""' -e '"journalVolume":""' -e '"id":""' -e '"ids":""' -e '"doi":""' -e '"doiUrl":""' semantic-scholar-raw/s2-corpus-clean.tmp > semantic-scholar-raw/s2-corpus-clean.json
!wc -l semantic-scholar-raw/s2-corpus-clean.json

86609 semantic-scholar-raw/s2-corpus-clean.json


Obtain a smaller sample from that data

In [29]:
!head -100 semantic-scholar-raw/s2-corpus-clean.json > semantic-scholar-raw/s2-corpus-slim.json

In [27]:
!head -1 semantic-scholar-raw/s2-corpus-slim.json | jq

{
  "entities": [
    "Eating",
    "Ethanol",
    "Isopropyl Alcohol 0.7 ML/ML Medicated Pad"
  ],
  "journalVolume": "31 4",
  "journalPages": "385-8",
  "pmid": "8879286v1",
  "year": 1996,
  "outCitations": [
    "40bb6636803d10626bfafa3dbf940fdac903d11c",
    "18ffb4ae9e0d8f0c43342c4c6b993f3ce34bd579",
    "827b793fbc923d3e068b2008dc02bb63e7069fb9",
    "b96cb6dd0bba1e9928eb6315e18339f667c6799f",
    "48431576480666445bdcb112533aa885d4ac1ad1",
    "1380d901dbce2dca8eba90501179485d4fe1e5d4",
    "4203ad4dc8530a92473e5576650eca84b51c9eee",
    "96b85a405b002e11199f967280ec9a9736543c86",
    "8d72d01b1b670c7bb65408e91d1705f0c95b1174",
    "7842615744a118a2d4a239787e9450c26c98c9f9",
    "6ed7439f9537fce3f82144843622b9c670a95d7e",
    "55787878e91f6efdf14e16438e733d80581f3bec",
    "e28468ccfa0ddce1d060d39d25305f6abab2eced",
    "5f6ba13f126408d25849c518bddf4812281ae970"
  ],
  "s2Url": "https://semanticscholar.org/paper/30aaf92bde9bf6cdc3dff237691fd5db55d4cb4a",
  "s2PdfUrl": "http://

The final modified JSON is accesible now from github

## A.2 Loading

First we will ensure that we start with a clean database

In [23]:
graph.run("MATCH (n) DETACH DELETE n").stats()

constraints_added: 0
constraints_removed: 0
contained_updates: False
indexes_added: 0
indexes_removed: 0
labels_added: 0
labels_removed: 0
nodes_created: 0
nodes_deleted: 0
properties_set: 0
relationships_created: 0
relationships_deleted: 0

We will be using apoc (https://github.com/neo4j-contrib/neo4j-apoc-procedures)

In [24]:
graph.run("RETURN apoc.version();").data()

[{'apoc.version()': '3.5.0.9'}]

In [25]:
graph.run("call dbms.components() yield name, versions, edition unwind versions as version return name, version, edition;").data()

[{'name': 'Neo4j Kernel', 'version': '3.5.15', 'edition': 'enterprise'}]

In [26]:
query = """
WITH $myjson AS url 
CALL apoc.load.json(url) 
YIELD value 
RETURN value 
LIMIT 1"""

graph.run(query, myjson = SC_JSON_GITHUB).data()

[{'value': {'doiUrl': 'https://doi.org/10.1007/BF02270828',
   'venue': 'Evolutionary Ecology',
   'journalName': 'Evolutionary Ecology',
   'sources': [],
   'year': 2005,
   'paperAbstract': 'I compare the relationship between bill size (depth) and body size among different taxa of seed-eating finches to test the hypothesis (Schluter, 1988a) that in habitats where seed-eating finches are vulnerable to predators, finches have larger bodies relative to their bill size. In support of this hypothesis, ground-foraging finches on continents (Emberizidae, Passeridae, Fringillidae), where predators are more common, have larger bodies relative to their bill sizes than do ground-foraging finches on islands (Emberizidae, Fringillidae). Ground-feeding finches on continents may also be more vulnerable to predators than tree- and shrub-foraging finches. As predicted, in North America, ground-feeding finches (Emberizinae) have larger bodies relative to their bill size than do tree- and shrub-foragi

Here, `"myjson"` is an alias for the path to the file, located in github

### Create Article instances

In [27]:
query_apoc = """
WITH $myjson AS url 
CALL apoc.load.json(url) 
YIELD value as row
MERGE (art:Article {
        title: row.title,
        id: row.id, doi: 
        row.doi, 
        doi_url:row.doiUrl,
        year: row.year})
MERGE (art)-[:PUBLISHED_IN {
        pages: row.journalPages}]->(vol:Volume {
            volume: row.journalVolume,
            year: row.year})-[:OF]->(jour:Journal {
            name: row.journalName})
FOREACH (author in row.authors |
    MERGE (auth:Author {name: author.name, id: author.ids})-[:AUTHORS]->(art)
)
FOREACH (k in row.entities |
    MERGE (art)-[:CONTAINS]->(kw: Keyword {name: k}))"""

graph.run(query_apoc, myjson = SC_JSON_GITHUB).stats()

constraints_added: 0
constraints_removed: 0
contained_updates: True
indexes_added: 0
indexes_removed: 0
labels_added: 1998
labels_removed: 0
nodes_created: 1998
nodes_deleted: 0
properties_set: 3089
relationships_created: 1898
relationships_deleted: 0

Some data is left to be simulated

### Faking citations

In [28]:
q_article_ids = "MATCH (n:Article) RETURN n.id"
article_ids = [v["n.id"] for v in graph.run(q_article_ids).data()]
n_articles = len(article_ids)
article_ids[:5]

['8fb9c95bf34a0f28dc05819cb4aada0cb94fe555',
 '5cfdb256b6ae968374469bd36702ed341cfe9485',
 '0fbe46932967ec0db80b18e70fa199fb652313ea',
 'c4062742b4e0d13cfa0e992fdf2cebf2eb71c415',
 'f218ce53248d756db61726985f73e6e8c109b3e2']

In [29]:
# https://neo4j.com/docs/driver-manual/1.7/sessions-transactions/#driver-transactions-transaction-functions
q_add_citation_rel_id = """MATCH (a:Article),(b:Article)
WHERE a.id = $id_a AND b.id = $id_b
MERGE p=(a)-[r:CITED_BY]->(b)
RETURN p"""

def add_citation_rel(driver, id_a, id_b):
    with driver.session() as session:
        # Caller for transactional unit of work
        return session.write_transaction(create_citation_rel, id_a, id_b)

# Simple implementation of the unit of work
def create_citation_rel(tx, id_a, id_b):
    return tx.run(q_add_citation_rel_id, id_a=id_a, id_b = id_b)

Add 500 relationships of type `cited_by`

In [30]:
random.seed(42)
# pick a sample of 500 papers, and make them be cited by other three papers at random
created_citations = []

for article in article_ids:
    for citation in random.sample(article_ids, random.randint(5, 50)):
        if article != citation: # they can't cite themselves
            _out = add_citation_rel(gdb, id_a=article, id_b=citation)
            created_citations.append(_out.data())
        else:
            print("Skipping: ", article, citation)

created_citations[:5]

Skipping:  8fb9c95bf34a0f28dc05819cb4aada0cb94fe555 8fb9c95bf34a0f28dc05819cb4aada0cb94fe555
Skipping:  8c5b2a75e5dfe1fc8c03c410ecfd2f44a05fa4a0 8c5b2a75e5dfe1fc8c03c410ecfd2f44a05fa4a0
Skipping:  2c86deb0e77a7fb9853472ec39749d209adb9440 2c86deb0e77a7fb9853472ec39749d209adb9440
Skipping:  eded0a1a0ff476c57fc3058ac893489c8e08415e eded0a1a0ff476c57fc3058ac893489c8e08415e
Skipping:  a09c196f66e8b5f44a7b734862a2a00e66d079a5 a09c196f66e8b5f44a7b734862a2a00e66d079a5
Skipping:  11781ea072a749847253639a9903e8b796d51e54 11781ea072a749847253639a9903e8b796d51e54
Skipping:  e3a9cc9bc08fa447463d46629aa24e2c78cdd87e e3a9cc9bc08fa447463d46629aa24e2c78cdd87e
Skipping:  04547565eae033cbd9826ca4e31b8cf00e2b2400 04547565eae033cbd9826ca4e31b8cf00e2b2400
Skipping:  a37913935bb44f54d54327927fc9dec2c4f68444 a37913935bb44f54d54327927fc9dec2c4f68444
Skipping:  d72eeacd4b5c694059ca575335e161c5beada79b d72eeacd4b5c694059ca575335e161c5beada79b
Skipping:  3bfb9955d7fb268294ec392aa6465a0ef95dce7e 3bfb9955d7fb268294

[[{'p': <Path start=<Node id=46 labels={'Article'} properties={'doi_url': 'https://doi.org/10.1007/s10926-005-9006-z', 'id': '8fb9c95bf34a0f28dc05819cb4aada0cb94fe555', 'title': 'Development of Work Stress Scale for Correctional Officers', 'year': 2005, 'doi': '10.1007/s10926-005-9006-z'}> end=<Node id=243 labels={'Article'} properties={'doi_url': 'https://doi.org/10.1089/sur.2016.107', 'id': 'acaa12d900e5a682f399162091845f8ad47a03f4', 'title': 'Topical Antimicrobials and the Open Surgical Wound.', 'year': 2016, 'doi': '10.1089/sur.2016.107'}> size=1>}],
 [{'p': <Path start=<Node id=46 labels={'Article'} properties={'doi_url': 'https://doi.org/10.1007/s10926-005-9006-z', 'id': '8fb9c95bf34a0f28dc05819cb4aada0cb94fe555', 'title': 'Development of Work Stress Scale for Correctional Officers', 'year': 2005, 'doi': '10.1007/s10926-005-9006-z'}> end=<Node id=72 labels={'Article'} properties={'doi_url': 'https://doi.org/10.1046/j.1525-1497.1999.00018.x', 'id': 'c4062742b4e0d13cfa0e992fdf2cebf

### Faking reviewers 

We will create a `REVIEWS` relationship between authors and articles

In [31]:
# delete relationships
graph.run("MATCH p=(:Author)-[r:REVIEWS]->(:Article) DELETE r RETURN COUNT(r)").stats()

constraints_added: 0
constraints_removed: 0
contained_updates: False
indexes_added: 0
indexes_removed: 0
labels_added: 0
labels_removed: 0
nodes_created: 0
nodes_deleted: 0
properties_set: 0
relationships_created: 0
relationships_deleted: 0

In [32]:
q_author_ids = "MATCH (n:Author) RETURN n.id"
author_ids = [v["n.id"] for v in graph.run(q_author_ids).data()]    
author_ids[:5]

[['3902390'], ['13413837'], ['24148377'], ['1792216'], ['2181642']]

#### Assign reviewers randomly to 1000 articles


In [33]:
# assign reviewers randomly to 1000 articles
random.seed(42)

q_add_reviewers = """MATCH (a:Article),(b:Author)
WHERE a.id = $article_id AND b.id = $author_id
AND NOT (b)-[:AUTHORS]->(a)
MERGE p=(b)-[:REVIEWS]->(a)
RETURN p"""

created_review_rel = []

with gdb.session() as session:
    for article_id in article_ids:
        for author_id in random.sample(author_ids, random.choice([2, 6])):
            _out = graph.run(q_add_reviewers, article_id=article_id, author_id=author_id)
            created_review_rel.append(_out.data())
            
created_review_rel[:5]

[[{'p': (Samuel C Mok)-[:REVIEWS {}]->(_46)}],
 [{'p': (Shaowei Ding)-[:REVIEWS {}]->(_46)}],
 [{'p': (Donald E Fry)-[:REVIEWS {}]->(_65)}],
 [{'p': (Beatriz Brignani)-[:REVIEWS {}]->(_65)}],
 [{'p': (Martin Kanning)-[:REVIEWS {}]->(_65)}]]

### Fake conferences and editions

In [34]:
# delete conference and editions
graph.run("MATCH (n:Conference) DETACH DELETE n RETURN COUNT(n) as deleted_conferences").data()

[{'deleted_conferences': 0}]

In [35]:
graph.run("MATCH (n:Edition) DETACH DELETE n RETURN COUNT(n) as deleted_editions").data()


[{'deleted_editions': 0}]

We will fake 600 editions, distributed between 100 conferences

In [6]:
random.seed(42)
N_CONFERENCES = 100

fake_conferences = [{
    "id": idx, 
    "name": fake.sentence(nb_words=6, variable_nb_words=True, ext_word_list=None)}
    for idx, _ in enumerate(range(100), start=1)]

fake_editions = [
    {
        "id": idx, 
        "venue": fake.company(),
        "city": fake.city(),
        "year": fake.year()}
    for idx, _ in enumerate(range(600), start=1)]

# assign 4 to 20 editions to each conference
for conference in fake_conferences:
    conference["editions"] = []
    for edition in random.sample(fake_editions, random.randint(4, 20)):
        conference["editions"].append(edition)

fake_conferences[:2]

[{'id': 1,
  'name': 'Agent every development say.',
  'editions': [{'id': 26,
    'venue': 'Allen, Reyes and Chase',
    'city': 'West Thomas',
    'year': '1995'},
   {'id': 282, 'venue': 'Salazar Inc', 'city': 'Dianeville', 'year': '1970'},
   {'id': 251,
    'venue': 'Cannon, Dunn and Wilson',
    'city': 'Lake Madelineborough',
    'year': '1972'},
   {'id': 229,
    'venue': 'Gonzalez, Osborn and White',
    'city': 'Johnsonchester',
    'year': '2018'},
   {'id': 143, 'venue': 'Harper Ltd', 'city': 'Justintown', 'year': '1977'},
   {'id': 105,
    'venue': 'Harrison, Gilbert and Bullock',
    'city': 'East Edwardside',
    'year': '2011'},
   {'id': 559,
    'venue': 'Walker-Williams',
    'city': 'Philipview',
    'year': '2010'}]},
 {'id': 2,
  'name': 'Beautiful instead ahead despite measure ago current.',
  'editions': [{'id': 433,
    'venue': 'Poole Ltd',
    'city': 'Ryanfort',
    'year': '1974'},
   {'id': 33, 'venue': 'Brooks Inc', 'city': 'Ashleefort', 'year': '1981'}

In [37]:
q_create_conference = "MERGE (c:Conference {id: $cid, name: $name}) RETURN c"
q_create_edition = "MERGE (e:Edition {id: $eid, conference_id: $cid, venue: $venue, city: $city, year: $year}) RETURN e"
q_merge_conference_and_edition = """MATCH (c:Conference), (e:Edition)
WHERE c.id = $cid AND e.id = $eid
MERGE p=(e)-[:OF]->(c)
RETURN p"""

created_conferences = []

with gdb.session() as session:
    for conference in fake_conferences:
        # create conference
        session.run(q_create_conference, cid = conference["id"], name = conference["name"])
        for edition in conference["editions"]:
            # create edition
            session.run(q_create_edition,
                        eid = edition["id"],
                        cid = conference["id"], 
                        venue = edition["venue"], 
                        city = edition["city"], 
                        year = edition["year"])
            # assign it to conference
            _out = session.run(q_merge_conference_and_edition, eid = edition["id"], cid = conference["id"])
            created_conferences.append(_out.data())

In [38]:
created_conferences[:5]

[[{'p': <Path start=<Node id=8266 labels={'Edition'} properties={'venue': 'Gordon PLC', 'conference_id': 1, 'id': 26, 'city': 'West Rebecca', 'year': '1986'}> end=<Node id=8264 labels={'Conference'} properties={'name': 'Benjamin Sullivan', 'id': 1}> size=1>}],
 [{'p': <Path start=<Node id=8267 labels={'Edition'} properties={'venue': 'Wright, Hill and Hoffman', 'conference_id': 1, 'id': 282, 'city': 'South Tiffany', 'year': '1982'}> end=<Node id=8264 labels={'Conference'} properties={'name': 'Benjamin Sullivan', 'id': 1}> size=1>}],
 [{'p': <Path start=<Node id=8268 labels={'Edition'} properties={'venue': 'Delgado-Mccann', 'conference_id': 1, 'id': 251, 'city': 'New Christine', 'year': '1979'}> end=<Node id=8264 labels={'Conference'} properties={'name': 'Benjamin Sullivan', 'id': 1}> size=1>}],
 [{'p': <Path start=<Node id=8272 labels={'Edition'} properties={'venue': 'White-Kelly', 'conference_id': 1, 'id': 229, 'city': 'West Cynthiastad', 'year': '1994'}> end=<Node id=8264 labels={'Con

### Assigning articles to editions

And we will assign articles randomly to 2 to 10 editions

In [39]:
fake_editions[:1]

[{'id': 1,
  'venue': 'Heath, Jones and Stafford',
  'city': 'Lake Louis',
  'year': '1991'}]

In [40]:
# assign each paper to 2-10 editions
random.seed(42)

q_link_to_conf = """MATCH (a:Article), (b:Edition)
WHERE a.id = $aid AND b.id = $eid
MERGE p=(a)-[r:PUBLISHED_IN]->(b)
RETURN p"""

created_art_ed_rel = []

with gdb.session() as session:
    for aid in article_ids:
        for edition in random.sample(fake_editions, random.randint(2, 10)):
            _out = session.run(q_link_to_conf, aid = aid, eid = edition["id"])
            created_art_ed_rel.append(_out.data())
            
created_art_ed_rel[:1]

[[{'p': <Path start=<Node id=46 labels={'Article'} properties={'doi_url': 'https://doi.org/10.1007/s10926-005-9006-z', 'id': '8fb9c95bf34a0f28dc05819cb4aada0cb94fe555', 'title': 'Development of Work Stress Scale for Correctional Officers', 'year': 2005, 'doi': '10.1007/s10926-005-9006-z'}> end=<Node id=8266 labels={'Edition'} properties={'venue': 'Gordon PLC', 'conference_id': 1, 'id': 26, 'city': 'West Rebecca', 'year': '1986'}> size=1>},
  {'p': <Path start=<Node id=46 labels={'Article'} properties={'doi_url': 'https://doi.org/10.1007/s10926-005-9006-z', 'id': '8fb9c95bf34a0f28dc05819cb4aada0cb94fe555', 'title': 'Development of Work Stress Scale for Correctional Officers', 'year': 2005, 'doi': '10.1007/s10926-005-9006-z'}> end=<Node id=9157 labels={'Edition'} properties={'venue': 'Gordon PLC', 'conference_id': 33, 'id': 26, 'city': 'West Rebecca', 'year': '1986'}> size=1>},
  {'p': <Path start=<Node id=46 labels={'Article'} properties={'doi_url': 'https://doi.org/10.1007/s10926-005-9

## A.3 Evolving the graph 



### Store the review and the approval sent by each reviewer

Since the queries in part B don't need this information, we can store the `review_decision` and the `review_text` as _edge_ attributes of the relation `REVIEWED_BY`.


Alternatively, and in the case these reviews were a requirement in part B, they can be implemented by creating a `Review` node, and linking it to articles, in a four step process.

1. Create a `Review` article for each `Article` that has reviewers (i.e. is connected to authors by `reviewed_by` edges). Each `Review` node will have an attribute named `review_contents´.
2. Attach review to the article
3. Attach reviewers to the review instance
4. Optional, delete the `reviewed_by` edges from the step 1.

In [41]:
with gdb.session() as session:
    out = session.run("MATCH p=(:Author)-[r:REVIEWS]->(:Article) return ID(r)")
    rel_ids = [r[0] for r in out.values()]
    
rel_ids[:5]

[1101993, 1101992, 1101994, 1101995, 1101996]

In [42]:
q_merge_review_attributes = """MATCH p=(:Author)-[r:REVIEWS]->(:Article)
WHERE ID(r) = $rel_id
SET r.review_accepted = $review_accepted, r.review_text = $review_text
RETURN r
"""

created_review_rel_attr = []

random.seed(42)

with gdb.session() as session:
    for rel_id in rel_ids:
        _out = session.run(q_merge_review_attributes, 
                    rel_id=rel_id,
                    review_accepted=fake.pybool(), 
                    review_text=fake.texts(nb_texts=1, max_nb_chars=500)[0])
        created_review_rel_attr.append(_out.data())

In [43]:
created_review_rel_attr[0]

[{'r': <Relationship id=1101993 nodes=(<Node id=6002 labels=set() properties={}>, <Node id=46 labels=set() properties={}>) type='REVIEWS' properties={'review_text': 'Despite minute agency idea sometimes least perform.\nAnything military but just maintain shake. Program nature see.\nYour check station form. Rule play century several cultural pattern. Hour among field well create.\nIndividual stay charge natural effort. A at teach enter lot. Despite trip close pull.\nProve speech garden road. Sure western election next threat sport.', 'review_accepted': False}>}]

### Store a reviewing policy for each Journal or Conference

This can also be implemented as an `review_policy_min_count` to the `Journal` and `Proceedings` labels. We will set all of them to 3.

In [44]:
with gdb.session() as session:
    journal_ids = session.run("MERGE (n:Journal) SET n.review_policy_min_count = 3 return n").values()
    proceeding_ids = session.run("MERGE (n:Conference) SET n.review_policy_min_count = 3 return n").values()

In [45]:
journal_ids[:5]

[[<Node id=1427 labels={'Journal'} properties={'review_policy_min_count': 3, 'name': 'Evolutionary Ecology'}>],
 [<Node id=1479 labels={'Journal'} properties={'review_policy_min_count': 3, 'name': 'Emergency medicine Australasia : EMA'}>],
 [<Node id=1499 labels={'Journal'} properties={'review_policy_min_count': 3, 'name': 'The American journal of surgical pathology'}>],
 [<Node id=1529 labels={'Journal'} properties={'review_policy_min_count': 3, 'name': 'Human Genetics'}>],
 [<Node id=1546 labels={'Journal'} properties={'review_policy_min_count': 3, 'name': 'International journal of cardiology'}>]]

In [46]:
proceeding_ids[:2]

[[<Node id=8264 labels={'Conference'} properties={'review_policy_min_count': 3, 'name': 'Benjamin Sullivan', 'id': 1}>],
 [<Node id=8278 labels={'Conference'} properties={'review_policy_min_count': 3, 'name': 'Michael Savage', 'id': 2}>]]

### Display the affiliation of the author to an organization or a company 

This can be solved by creating an `affiliation_institution_type` attribute, and also an `affiliation_institution_name`. Note that we could have created these as nodes, but the queries from part B do not refer to this information.


In [47]:
q_set_affiliations = """MATCH (n:Author)
WHERE n.id = $author_id
SET n.affiliation_institution_type = $affiliation_institution_type, n.affiliation_institution_name = $affiliation_institution_name
RETURN n
"""

institution_types = ['University', 'Company', 'NGO']
created_affiliations = []

random.seed(42)

with gdb.session() as session:
    for author_id in author_ids:
        _out = session.run(q_set_affiliations, 
                    author_id=author_id,
                    affiliation_institution_name=fake.company(), 
                    affiliation_institution_type=random.choice(institution_types))        
        created_affiliations.append(_out.values())

In [48]:
created_affiliations

[[[<Node id=4695 labels={'Author'} properties={'name': 'Craig W. Benkman', 'affiliation_institution_name': 'Sanders, Sherman and Brown', 'id': ['3902390'], 'affiliation_institution_type': 'NGO'}>]],
 [[<Node id=4712 labels={'Author'} properties={'name': 'Liz Crowe', 'affiliation_institution_name': 'Lewis-Olsen', 'id': ['13413837'], 'affiliation_institution_type': 'University'}>]],
 [[<Node id=4749 labels={'Author'} properties={'name': 'Erin R King', 'affiliation_institution_name': 'George Group', 'id': ['24148377'], 'affiliation_institution_type': 'University'}>]],
 [[<Node id=4752 labels={'Author'} properties={'name': 'Celestine S Tung', 'affiliation_institution_name': 'Neal Group', 'id': ['1792216'], 'affiliation_institution_type': 'NGO'}>]],
 [[<Node id=4758 labels={'Author'} properties={'name': 'Yvonne T M Tsang', 'affiliation_institution_name': 'Butler, Lopez and Bates', 'id': ['2181642'], 'affiliation_institution_type': 'Company'}>]],
 [[<Node id=4764 labels={'Author'} properties